# Market Data Integration with CustomData

This notebook demonstrates how to fetch real market data from Yahoo Finance and integrate it into Zipline using the CustomData functionality.

## What You'll Learn

1. Fetching historical market data from Yahoo Finance
2. Creating a CustomData database for market prices
3. Inserting and managing market data
4. Building pipelines with real market data
5. Performing technical analysis and visualization

## Prerequisites

First, we need to install `yfinance` to fetch data from Yahoo Finance:

In [ ]:
# Install yfinance if not already installed
!pip install yfinance -q

## Step 1: Import Required Libraries

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from datetime import datetime, timedelta

# Zipline imports
from zipline.pipeline.data import create_custom_db, insert_custom_data, from_db
from zipline.pipeline.data import query_custom_data, list_custom_dbs, get_custom_db_info
from zipline.pipeline import Pipeline
from zipline.pipeline.engine import SimplePipelineEngine
from zipline.pipeline.loaders.custom_db_loader import DatabaseCustomDataLoader
from zipline.utils.calendar_utils import get_calendar

warnings.filterwarnings('ignore')

# Set up matplotlib for better plots
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ All libraries imported successfully!")

## Step 2: Define Stocks and Date Range

Let's fetch data for some popular tech stocks:

In [ ]:
# Define our universe of stocks
tickers = {
    'AAPL': 'Apple Inc.',
    'MSFT': 'Microsoft Corporation',
    'GOOGL': 'Alphabet Inc.',
    'AMZN': 'Amazon.com Inc.',
    'TSLA': 'Tesla Inc.',
    'NVDA': 'NVIDIA Corporation',
    'META': 'Meta Platforms Inc.',
    'JPM': 'JPMorgan Chase & Co.',
    'V': 'Visa Inc.',
    'WMT': 'Walmart Inc.',
}

# Date range for historical data
start_date = '2022-01-01'
end_date = '2023-12-31'

# Database directory
db_dir = '/data/custom_databases'

print(f"Stocks to fetch: {len(tickers)}")
print(f"Date range: {start_date} to {end_date}")
print(f"Database directory: {db_dir}")

## Step 3: Fetch Market Data from Yahoo Finance

We'll download historical OHLCV (Open, High, Low, Close, Volume) data:

In [ ]:
print("Downloading market data from Yahoo Finance...\n")

all_data = []
ticker_to_sid = {}  # Map ticker symbols to numeric SIDs
sid_to_ticker = {}  # Reverse mapping

for sid, (ticker, company_name) in enumerate(tickers.items(), start=1):
    print(f"  [{sid}/{len(tickers)}] Fetching {ticker} ({company_name})...", end=" ")
    
    try:
        # Download data
        df = yf.download(ticker, start=start_date, end=end_date, progress=False)
        
        if df.empty:
            print("❌ No data available")
            continue
        
        # Add sid column
        df['sid'] = sid
        df['ticker'] = ticker
        
        all_data.append(df)
        ticker_to_sid[ticker] = sid
        sid_to_ticker[sid] = ticker
        
        print(f"✓ {len(df)} days")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print(f"\n✓ Downloaded data for {len(all_data)} stocks")
print(f"  Total data points: {sum(len(df) for df in all_data):,}")

# Display sample data
if all_data:
    print("\nSample data for", all_data[0]['ticker'].iloc[0] + ":")
    print(all_data[0][['Open', 'High', 'Low', 'Close', 'Volume']].head())

## Step 4: Create CustomData Database

Now we'll create a database to store this market data:

In [ ]:
print("Creating CustomData database for market prices...\n")

# Create database with OHLCV columns
create_custom_db(
    'yahoo-market-data',
    columns={
        'open': float,
        'high': float,
        'low': float,
        'close': float,
        'adj_close': float,  # Adjusted close (for splits/dividends)
        'volume': float,
    },
    bar_size='1d',
    db_dir=db_dir,
)

print("✓ Database 'yahoo-market-data' created successfully!")

# Display database info
db_info = get_custom_db_info('yahoo-market-data', db_dir=db_dir)
print("\nDatabase Information:")
for key, value in db_info.items():
    print(f"  {key}: {value}")

## Step 5: Insert Market Data into Database

Convert the data to the required format and insert it:

In [ ]:
print("Inserting market data into database...\n")

for i, df in enumerate(all_data, 1):
    sid = df['sid'].iloc[0]
    ticker = df['ticker'].iloc[0]
    
    print(f"  [{i}/{len(all_data)}] Inserting {ticker} (SID {sid})...", end=" ")
    
    try:
        # Create MultiIndex DataFrame (field, sid)
        data = pd.DataFrame({
            ('open', sid): df['Open'],
            ('high', sid): df['High'],
            ('low', sid): df['Low'],
            ('close', sid): df['Close'],
            ('adj_close', sid): df['Adj Close'],
            ('volume', sid): df['Volume'],
        })
        
        # Set MultiIndex
        data.columns = pd.MultiIndex.from_tuples(data.columns, names=['field', 'sid'])
        
        # Insert into database
        insert_custom_data(
            'yahoo-market-data',
            data,
            mode='update',  # Use update mode to handle overlapping dates
            db_dir=db_dir,
        )
        
        print(f"✓ {len(data)} rows")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n✓ All market data inserted successfully!")

# Update database info
db_info = get_custom_db_info('yahoo-market-data', db_dir=db_dir)
print(f"\nTotal rows in database: {db_info.get('row_count', 'N/A'):,}")

## Step 6: Query and Verify Data

Let's query the database to verify our data was inserted correctly:

In [ ]:
# Query data for a specific stock and date range
test_ticker = 'AAPL'
test_sid = ticker_to_sid[test_ticker]

print(f"Querying data for {test_ticker} (SID {test_sid})...\n")

result = query_custom_data(
    'yahoo-market-data',
    start_date='2023-01-01',
    end_date='2023-01-31',
    sids=[test_sid],
    columns=['close', 'volume'],
    db_dir=db_dir,
)

print(f"Retrieved {len(result)} rows\n")
print(result.head(10))

# Calculate and display basic statistics
print(f"\n{test_ticker} Statistics (Jan 2023):")
print(f"  Close: ${result['close'].mean():.2f} avg, ${result['close'].min():.2f} - ${result['close'].max():.2f}")
print(f"  Volume: {result['volume'].mean():,.0f} avg")

## Step 7: Create Pipeline with Market Data

Now let's use this data in a Zipline Pipeline:

In [ ]:
from zipline.pipeline.factors import CustomFactor, Returns

print("Loading market data into Pipeline...\n")

# Load the dataset
MarketData = from_db('yahoo-market-data', db_dir=db_dir)

print("✓ MarketData dataset loaded")
print(f"  Available columns: {', '.join([c for c in dir(MarketData) if not c.startswith('_')])}")

# Create custom factors
class DailyReturn(CustomFactor):
    """Calculate daily return"""
    inputs = [MarketData.close]
    window_length = 2
    
    def compute(self, today, assets, out, close):
        out[:] = (close[-1] - close[-2]) / close[-2]

class Volatility(CustomFactor):
    """Calculate 20-day volatility"""
    inputs = [MarketData.close]
    window_length = 20
    
    def compute(self, today, assets, out, close):
        # Calculate daily returns
        returns = np.diff(close, axis=0) / close[:-1]
        # Calculate standard deviation
        out[:] = np.std(returns, axis=0)

class PriceRange(CustomFactor):
    """Calculate daily price range (high - low)"""
    inputs = [MarketData.high, MarketData.low]
    window_length = 1
    
    def compute(self, today, assets, out, high, low):
        out[:] = high[-1] - low[-1]

print("\n✓ Custom factors defined")

## Step 8: Build and Run Pipeline

Create a pipeline with various technical indicators:

In [ ]:
from zipline.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

print("Building pipeline...\n")

# Define the pipeline
pipeline = Pipeline(
    columns={
        # Price data
        'close': MarketData.close.latest,
        'high': MarketData.high.latest,
        'low': MarketData.low.latest,
        'volume': MarketData.volume.latest,
        
        # Moving averages
        'sma_10': SimpleMovingAverage(inputs=[MarketData.close], window_length=10),
        'sma_50': SimpleMovingAverage(inputs=[MarketData.close], window_length=50),
        
        # Technical indicators
        'daily_return': DailyReturn(),
        'volatility_20d': Volatility(),
        'price_range': PriceRange(),
        
        # Trading signals
        'high_volume': MarketData.volume.latest > MarketData.volume.latest.percentile(75),
        'above_sma50': MarketData.close.latest > SimpleMovingAverage(inputs=[MarketData.close], window_length=50),
    }
)

print("✓ Pipeline created with", len(pipeline.columns), "columns")

## Step 9: Set Up Pipeline Engine

Configure the engine to run our pipeline:

In [ ]:
print("Setting up Pipeline engine...\n")

# Get trading calendar
trading_calendar = get_calendar('NYSE')

# Create loader for our dataset
loader = DatabaseCustomDataLoader(
    dataset=MarketData,
    db_path=f"{db_dir}/yahoo-market-data.db",
)

# Create engine
def get_loader(column):
    if column.dataset == MarketData:
        return loader
    raise ValueError(f"No loader for {column}")

engine = SimplePipelineEngine(
    get_loader=get_loader,
    asset_finder=None,  # We're using SIDs directly
    default_domain=None,
)

print("✓ Pipeline engine ready")

## Step 10: Run Pipeline and Analyze Results

Execute the pipeline for a specific date range:

In [ ]:
print("Running pipeline...\n")

# Define date range for analysis
analysis_start = pd.Timestamp('2023-06-01', tz='UTC')
analysis_end = pd.Timestamp('2023-06-30', tz='UTC')

# Get valid trading days
trading_days = trading_calendar.sessions_in_range(analysis_start, analysis_end)

print(f"Analysis period: {analysis_start.date()} to {analysis_end.date()}")
print(f"Trading days: {len(trading_days)}\n")

# Run pipeline
results = engine.run_pipeline(
    pipeline,
    start_date=analysis_start,
    end_date=analysis_end,
)

print(f"✓ Pipeline executed successfully!")
print(f"  Result shape: {results.shape}")
print(f"  Columns: {len(results.columns)}\n")

# Display sample results
print("Sample results:")
print(results.head(10))

## Step 11: Analysis and Visualization

Let's analyze the results and create visualizations:

In [ ]:
# Analyze by asset (SID)
print("\n=== Analysis by Stock ===\n")

for sid, ticker in sorted(sid_to_ticker.items()):
    if sid in results.index.get_level_values(1):
        stock_data = results.xs(sid, level=1)
        
        print(f"{ticker} (SID {sid}):")
        print(f"  Avg Close: ${stock_data['close'].mean():.2f}")
        print(f"  Avg Daily Return: {stock_data['daily_return'].mean()*100:.2f}%")
        print(f"  Volatility (20d): {stock_data['volatility_20d'].mean():.4f}")
        print(f"  Days above SMA50: {stock_data['above_sma50'].sum()}")
        print(f"  High volume days: {stock_data['high_volume'].sum()}")
        print()

In [ ]:
# Plot 1: Price trends with moving averages
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Market Data Analysis - June 2023', fontsize=16, fontweight='bold')

# Select 4 stocks to plot
plot_tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA']

for idx, ticker in enumerate(plot_tickers):
    if ticker not in ticker_to_sid:
        continue
        
    sid = ticker_to_sid[ticker]
    if sid not in results.index.get_level_values(1):
        continue
    
    ax = axes[idx // 2, idx % 2]
    stock_data = results.xs(sid, level=1)
    
    # Plot price and moving averages
    ax.plot(stock_data.index, stock_data['close'], label='Close', linewidth=2)
    ax.plot(stock_data.index, stock_data['sma_10'], label='SMA 10', linestyle='--', alpha=0.7)
    ax.plot(stock_data.index, stock_data['sma_50'], label='SMA 50', linestyle='--', alpha=0.7)
    
    ax.set_title(f'{ticker} - Price & Moving Averages', fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price ($)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Rotate x-axis labels
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("✓ Price trends plotted")

In [ ]:
# Plot 2: Daily returns and volatility
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('Returns and Volatility Analysis', fontsize=16, fontweight='bold')

# Calculate average returns and volatility per stock
stock_stats = []
for sid, ticker in sorted(sid_to_ticker.items()):
    if sid in results.index.get_level_values(1):
        stock_data = results.xs(sid, level=1)
        stock_stats.append({
            'ticker': ticker,
            'avg_return': stock_data['daily_return'].mean() * 100,
            'volatility': stock_data['volatility_20d'].mean(),
        })

stats_df = pd.DataFrame(stock_stats)

# Plot 1: Average daily returns
ax1 = axes[0]
colors = ['green' if x > 0 else 'red' for x in stats_df['avg_return']]
ax1.barh(stats_df['ticker'], stats_df['avg_return'], color=colors, alpha=0.7)
ax1.set_xlabel('Average Daily Return (%)')
ax1.set_title('Average Daily Returns', fontweight='bold')
ax1.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
ax1.grid(True, alpha=0.3, axis='x')

# Plot 2: Volatility
ax2 = axes[1]
ax2.barh(stats_df['ticker'], stats_df['volatility'], color='orange', alpha=0.7)
ax2.set_xlabel('20-Day Volatility')
ax2.set_title('Price Volatility', fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("✓ Returns and volatility plotted")

In [ ]:
# Plot 3: Volume analysis
fig, ax = plt.subplots(figsize=(16, 6))

# Plot volume for selected stocks
for ticker in plot_tickers:
    if ticker not in ticker_to_sid:
        continue
        
    sid = ticker_to_sid[ticker]
    if sid not in results.index.get_level_values(1):
        continue
    
    stock_data = results.xs(sid, level=1)
    ax.plot(stock_data.index, stock_data['volume'] / 1e6, label=ticker, linewidth=2, marker='o', markersize=4)

ax.set_title('Trading Volume - June 2023', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Volume (Millions)')
ax.legend()
ax.grid(True, alpha=0.3)
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("✓ Volume analysis plotted")

## Step 12: Risk-Return Analysis

Create a risk-return scatter plot:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

# Plot each stock
for _, row in stats_df.iterrows():
    ax.scatter(row['volatility'], row['avg_return'], s=200, alpha=0.6)
    ax.annotate(row['ticker'], (row['volatility'], row['avg_return']), 
                fontsize=10, fontweight='bold', ha='center')

ax.set_xlabel('Volatility (Risk)', fontsize=12)
ax.set_ylabel('Average Daily Return (%)', fontsize=12)
ax.set_title('Risk-Return Profile - June 2023', fontsize=14, fontweight='bold')
ax.axhline(y=0, color='black', linestyle='--', linewidth=0.5)
ax.grid(True, alpha=0.3)

# Add quadrant labels
xlim = ax.get_xlim()
ylim = ax.get_ylim()
ax.text(xlim[1]*0.9, ylim[1]*0.9, 'High Risk\nHigh Return', 
        ha='right', va='top', fontsize=10, alpha=0.5, style='italic')
ax.text(xlim[0]*1.1, ylim[1]*0.9, 'Low Risk\nHigh Return', 
        ha='left', va='top', fontsize=10, alpha=0.5, style='italic')
ax.text(xlim[1]*0.9, ylim[0]*1.1, 'High Risk\nLow Return', 
        ha='right', va='bottom', fontsize=10, alpha=0.5, style='italic')
ax.text(xlim[0]*1.1, ylim[0]*1.1, 'Low Risk\nLow Return', 
        ha='left', va='bottom', fontsize=10, alpha=0.5, style='italic')

plt.tight_layout()
plt.show()

print("✓ Risk-return analysis complete")

## Step 13: Trading Signals

Identify trading opportunities based on our pipeline signals:

In [ ]:
print("\n=== Trading Signals ===\n")

# Get the most recent date
latest_date = results.index.get_level_values(0).max()
latest_data = results.xs(latest_date, level=0)

print(f"Analysis date: {latest_date.date()}\n")

# Identify stocks above SMA50 (bullish signal)
bullish_stocks = latest_data[latest_data['above_sma50'] == True]
print("📈 Bullish Signals (Price > SMA50):")
for sid in bullish_stocks.index:
    if sid in sid_to_ticker:
        ticker = sid_to_ticker[sid]
        price = bullish_stocks.loc[sid, 'close']
        sma50 = bullish_stocks.loc[sid, 'sma_50']
        print(f"  {ticker}: ${price:.2f} (SMA50: ${sma50:.2f}, +{((price/sma50-1)*100):.2f}%)")

# Identify high volume stocks
high_vol_stocks = latest_data[latest_data['high_volume'] == True]
print(f"\n📊 High Volume Stocks ({len(high_vol_stocks)}):")
for sid in high_vol_stocks.index:
    if sid in sid_to_ticker:
        ticker = sid_to_ticker[sid]
        volume = high_vol_stocks.loc[sid, 'volume']
        print(f"  {ticker}: {volume:,.0f} shares")

# Identify high performers (positive return + above SMA50)
strong_stocks = latest_data[
    (latest_data['above_sma50'] == True) & 
    (latest_data['daily_return'] > 0)
]
print(f"\n🚀 Strong Momentum ({len(strong_stocks)} stocks):")
for sid in strong_stocks.index:
    if sid in sid_to_ticker:
        ticker = sid_to_ticker[sid]
        ret = strong_stocks.loc[sid, 'daily_return'] * 100
        print(f"  {ticker}: +{ret:.2f}% today")

## Step 14: Summary Statistics

Generate comprehensive statistics for the analysis period:

In [ ]:
print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60 + "\n")

print(f"Analysis Period: {analysis_start.date()} to {analysis_end.date()}")
print(f"Number of Stocks: {len(sid_to_ticker)}")
print(f"Trading Days: {len(trading_days)}")
print(f"Total Data Points: {len(results):,}\n")

# Portfolio-level statistics
print("Portfolio Metrics:")
print(f"  Average Daily Return: {results['daily_return'].mean()*100:.3f}%")
print(f"  Return Std Dev: {results['daily_return'].std()*100:.3f}%")
print(f"  Average Volatility: {results['volatility_20d'].mean():.4f}")
print(f"  Average Price Range: ${results['price_range'].mean():.2f}")

# Best and worst performers
avg_returns_by_stock = results.groupby(level=1)['daily_return'].mean() * 100
avg_returns_by_stock = avg_returns_by_stock.sort_values(ascending=False)

print("\nBest Performers:")
for sid in avg_returns_by_stock.head(3).index:
    if sid in sid_to_ticker:
        ticker = sid_to_ticker[sid]
        ret = avg_returns_by_stock[sid]
        print(f"  {ticker}: {ret:+.3f}%")

print("\nWorst Performers:")
for sid in avg_returns_by_stock.tail(3).index:
    if sid in sid_to_ticker:
        ticker = sid_to_ticker[sid]
        ret = avg_returns_by_stock[sid]
        print(f"  {ticker}: {ret:+.3f}%")

print("\n" + "="*60)
print("✓ Analysis Complete!")
print("="*60)

## Next Steps

Now that you have real market data integrated with Zipline, you can:

1. **Build Trading Strategies**: Use the Pipeline API to create complex trading algorithms
2. **Backtest**: Test your strategies against historical data
3. **Add More Data**: Integrate fundamental data, sentiment analysis, or alternative data
4. **Real-time Updates**: Set up scheduled tasks to update the database daily
5. **Portfolio Optimization**: Use the data for portfolio construction and risk management

### Resources

- [Zipline Pipeline Documentation](https://zipline.ml4trading.io/api-reference.html#pipeline-api)
- [CustomData Documentation](../docs/CUSTOM_DATA.md)
- [Database Storage Guide](../docs/CUSTOM_DATA_DATABASE.md)
- [Yahoo Finance Python API](https://pypi.org/project/yfinance/)

### Tips for Production Use

1. **API Rate Limits**: Be aware of Yahoo Finance rate limits; consider caching
2. **Data Quality**: Always validate data for missing values, splits, and corporate actions
3. **Update Schedule**: Set up daily data updates after market close
4. **Backup**: Regularly backup your custom databases
5. **Error Handling**: Implement robust error handling for data fetching

### Example Update Script

```python
# update_market_data.py
import yfinance as yf
from zipline.pipeline.data import insert_custom_data
from datetime import datetime, timedelta

def update_daily():
    """Update database with yesterday's data"""
    yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    
    for sid, ticker in sid_to_ticker.items():
        df = yf.download(ticker, start=yesterday, end=datetime.now())
        # Format and insert...
        
if __name__ == '__main__':
    update_daily()
```

You can schedule this with cron (Linux/Mac) or Task Scheduler (Windows).